# Scrub
Create steps in pipeline for processing data.

Issues that were discovered during EDA will be addressed here.

## Transformers
Quick article that describes the process for creating custom sklearn type transformers:

https://towardsdatascience.com/custom-transformers-and-ml-data-pipelines-with-python-20ea2a7adb65

### Numerical Features
Feature | Pipeline Steps | Hyperparameters
---|---|---
gps_height|<code>GPSHeightImputer()</code><br><code>StandardScaler()</code>|
latitude, longitude|<code>LatLongImputer()</code><br><code>StandardScaler()</code>|
population|<code>PopulationImputer()</code><br><code>StandardScaler()</code>|
construction_year|<code>ConstructionYearTransformer()</code><br><code>StandardScaler()</code>|

### Categorical Features
Feature | Pipeline Steps | Hyperparameters
---|---|---
funder<br>installer<br>lga<br>ward<br>scheme_management<br>scheme_name|<code>SimpleImputer()</code><br>HighCardTransformer()</code><br><code>StandardScaler()</code>|
district_code<br>region|<code>SimpleImputer()</code><br><code>DistrictCodeMerge()</code><br><code>HighCardTransformer()</code><br><code>StandardScaler()</code>|
basin<br>public_meeting<br>permit<br>management<br>management_group<br>payment<br>water_quality<br>quantity<br>source<br>source_class<br>waterpoint_type|<code>SimpleImputer()</code><br><code>OneHotEncoder()</code>|
extraction_type<br>extraction_type_group<br>extraction_type_class|<code>SimpleImputer()</code><br><code>ExtractionMerge()</code><br><code>OneHotEncoder()</code>|

## Implementation

In [6]:
# import libraries
from sklearn.base import BaseEstimator, TransformerMixin
import pickle
import math
import pandas as pd
import numpy as np

In [7]:
X = pd.read_csv('../data/raw/X_train.csv')
y = pd.read_csv('../data/raw/y_train.csv')

df = X.set_index('id').join(y.set_index('id')).reset_index()

In [8]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


### GPSHeightImputer()

In [9]:
class GPSHeightImputer(BaseEstimator, TransformerMixin):
    def __init__(self,init_radius=0.1,increment_radius=0.3,method = 'custom'): 
        self.column_names = [] 
        self.init_radius = init_radius
        self.increment_radius = increment_radius
        self.method = method

    
    def __get_subset_records(self, latitude, longitude, df, radius):
        latitude_from = latitude - radius
        latitude_to = latitude + radius
        longitude_from = longitude - radius
        longitude_to = longitude + radius
        
        df_temp = df[(df['latitude'] >= latitude_from) & (df['latitude'] <= latitude_to) & 
                  (df['longitude'] >= longitude_from) & (df['longitude'] <= longitude_to)]
        return df_temp
       

    def fit(self, X, y=None):
        if self.method == 'custom':
            X['gps_height'] = X['gps_height'].astype(float)
            X['latitude'] = X['latitude'].astype(float)
            X['longitude'] = X['longitude'].astype(float)
            self.df = X[X['gps_height'] != 0]
        elif self.method == 'median':
            X['gps_height'] = X['gps_height'].astype(float)
            #X['gps_height'] = X['gps_height'].fillna(0)
            self.median = np.median(list(X[X['gps_height'] != 0]['gps_height']))

            if math.isnan(self.median):
                self.median = 0
        elif self.method == 'mean':
            X['gps_height'] = X['gps_height'].astype(float)
            #X['gps_height'] = X['gps_height'].fillna(0)
            self.mean = np.mean(list(X[X['gps_height'] != 0]['gps_height']))
            if math.isnan(self.mean):
                self.mean = 0

        self.column_names = X.columns
        return self
        
    def transform(self,X):
        if self.method == 'custom':
            X['gps_height'] = X['gps_height'].astype(float)
            X['latitude'] = X['latitude'].astype(float)
            X['longitude'] = X['longitude'].astype(float)
            gps_height_transformed = []
            for latitude, longitude, gps_height in \
                zip(X['latitude'],X['longitude'],X['gps_height']):
                radius = self.init_radius
                if gps_height == 0:
                    gps_height_temp = gps_height
                    while gps_height_temp == 0 and radius <= 2:
                        df_temp = self.__get_subset_records\
                                  (latitude,longitude,self.df,radius)

                        gps_height_temp = np.mean(df_temp[df_temp['gps_height']!=0]\
                                                  ['gps_height'])
                        if math.isnan(gps_height_temp):
                            gps_height_temp = 0 
                        radius = self.increment_radius + radius
                else:
                    gps_height_temp = gps_height
                gps_height_transformed.append(gps_height_temp)
            X['gps_height'] = gps_height_transformed
            #self.column_names = list(X.columns)
        elif self.method == 'median':
            gps_height = np.array(list(X['gps_height']))
            gps_height[gps_height == 0] = self.median
            #self.column_names = list(X.columns)
            #return X[['latitude','longitude','gps_height']]
            X['gps_height'] = gps_height
        elif self.method == 'mean':
            gps_height = np.array(list(X['gps_height']))
            gps_height[gps_height == 0] = self.mean
            #self.column_names = list(X.columns)
            #return X[['latitude','longitude','gps_height']]
            X['gps_height'] = gps_height

        self.column_names = X.columns
        X['gps_height'] = X['gps_height'].astype(float)
        X['gps_height'] = X['gps_height'].fillna(0)
        return X

### Test

In [10]:
ghi_test_df = df[['latitude', 'longitude', 'gps_height', 'population']].copy()

ghi = GPSHeightImputer(method='custom').fit(ghi_test_df)
ghi.transform(ghi_test_df)

,latitude,longitude,gps_height,population
0,-9.856322,34.938093,1390.000000,109
1,-2.147466,34.698766,1399.000000,280
2,-3.821329,37.460664,686.000000,250
3,-11.155298,38.486161,263.000000,58
4,-1.825359,31.130847,1392.266667,0
...,...,...,...,...
59395,-3.253847,37.169807,1210.000000,125
59396,-9.070629,35.249991,1212.000000,56
59397,-8.750434,34.017087,1071.333333,0
59398,-6.378573,35.861315,961.387097,0


### LatLongImputer()

In [107]:
# Here will be our strategy to handle gps_height:
# In fit() just save the input data as a data frame \
# with gps_height, lat, long, and gps_height > 0
# In transform(), if gps_height == 0, then
# start at 0.1 radius, and check if there are any non-zero gps_instances.
# If yes, get the average, else, increment search radius
# by 0.3 (0.1 increase corresponds to 11km approximately)
# If nothing is found within an increment of 2, then just ignore.

class LatLongImputer(BaseEstimator, TransformerMixin):
    def __init__(self, method='custom'):
        self.column_names = []
        self.method = method
        self.df = pd.DataFrame()
        self.long_mean_map = {}
        self.lat_mean_map = {}
        pass

    def __generate_mean_maps(self):
        temp_df = self.df[(self.df['latitude'] != -2e-08) & (self.df['longitude'] != 0)]
        for geo in ['ward', 'region', 'basin']:
            self.long_mean_map[geo] = dict(zip(temp_df.groupby(geo)['longitude'].mean(
            ).keys(), temp_df.groupby(geo)['longitude'].mean().values))
            self.lat_mean_map[geo] = dict(zip(temp_df.groupby(geo)['latitude'].mean(
            ).keys(), temp_df.groupby(geo)['latitude'].mean().values))

    def fit(self, X, y=None):
        if self.method == 'mean':
            # find mean of all non-zero values
            self.mean_lat = np.mean(X[X['latitude'] != -2e-08]['latitude'])
            self.mean_long = np.mean(X[X['longitude'] != 0]['longitude'])
        elif self.method == 'median':
            # find median of all non-zero values
            self.median_lat = np.median(X[X['latitude'] != -2e-08]['latitude'])
            self.median_long = np.median(X[X['longitude'] != 0]['longitude'])
        elif self.method == 'custom':
            self.df = X
            self.__generate_mean_maps()
        self.column_names = ['latitude', 'longitude', 'gps_height']
        return self

    def transform(self, X):
        if self.method == 'mean':
            X['latitude'].replace(-2e-08, self.mean_lat, inplace=True)
            X['longitude'].replace(0, self.mean_long, inplace=True)
        elif self.method == 'median':
            X['latitude'].replace(-2e-08, self.median_lat, inplace=True)
            X['longitude'].replace(0, self.median_long, inplace=True)
        elif self.method == 'custom':
            X[(X['latitude'] == -2e-08)]['latitude'] = X['latitude'].map(self.lat_mean_map)
            X[(X['longitude'] == 0)]['longitude'] = X['longitude'].map(self.long_mean_map)
        self.column_names = X.columns
        return X

### Test

In [108]:
lli_test_df = df[['latitude', 'longitude', 'gps_height', 'population', 'ward', 'region', 'basin']].copy()

lli = LatLongImputer(method='custom').fit(lli_test_df)
lli.transform(lli_test_df)

<ipython-input-107-aa4d196497a4>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[(X['latitude'] == -2e-08)]['latitude'] = X['latitude'].map(self.lat_mean_map)
<ipython-input-107-aa4d196497a4>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[(X['longitude'] == 0)]['longitude'] = X['longitude'].map(self.long_mean_map)


,latitude,longitude,gps_height,population,ward,region,basin
0,-9.856322,34.938093,1390,109,Mundindi,Iringa,Lake Nyasa
1,-2.147466,34.698766,1399,280,Natta,Mara,Lake Victoria
2,-3.821329,37.460664,686,250,Ngorika,Manyara,Pangani
3,-11.155298,38.486161,263,58,Nanyumbu,Mtwara,Ruvuma / Southern Coast
4,-1.825359,31.130847,0,0,Nyakasimbi,Kagera,Lake Victoria
...,...,...,...,...,...,...,...
59395,-3.253847,37.169807,1210,125,Masama Magharibi,Kilimanjaro,Pangani
59396,-9.070629,35.249991,1212,56,Ikondo,Iringa,Rufiji
59397,-8.750434,34.017087,0,0,Chimala,Mbeya,Rufiji
59398,-6.378573,35.861315,0,0,Mvumi Makulu,Dodoma,Rufiji


In [104]:
df['latitude']

0        -9.856322
1        -2.147466
2        -3.821329
3       -11.155298
4        -1.825359
           ...    
59395    -3.253847
59396    -9.070629
59397    -8.750434
59398    -6.378573
59399    -6.747464
Name: latitude, Length: 59400, dtype: float64

### PopulationImputer()

In [13]:
# will work the same way as the gps imputer. 
class PopulationImputer(BaseEstimator, TransformerMixin):
    def __init__(self, method='custom'):
        self.columns_names = []
        self.method = method
        self.df = pd.DataFrame()
        
    def fit(self, X, y=None):
        if self.method == 'mean':
            self.mean = np.mean(X[X['population'] > 0]['population'])
        elif self.method == 'median':
            self.median = np.median(X[X['population'] > 0]['population'])
        elif self.method == 'custom':
            self.df['population'] = X[X['population'] > 0]['population']
        
        self.column_names = ['latitude', 'longitude', 'population']
        return self
    
    def transform(self, X):
        X.fillna(0, inplace=True)
        if self.method == 'mean':
            X['population'].replace(0, self.mean, inplace=True)
        elif self.method == 'median':
            X['population'].replace(0, self.median, inplace=True)
        elif self.method == 'custom':
            pass
        self.column_names = ['latitude', 'longitude', 'population']
        return X[['latitude', 'longitude', 'population']]

### Test

In [14]:
pi_test_df = df[['population', 'latitude', 'longitude']].copy()

pi = PopulationImputer(method='median').fit(pi_test_df)
pi.transform(pi_test_df).sample(5)

,latitude,longitude,population
28646,-4.147836,30.124768,1200
44130,-7.241450,39.211333,400
58080,-9.567987,39.493369,568
8945,-9.782846,35.241147,260
39797,-3.402346,36.951253,120


### ConstructionYearTransformer()

In [15]:
class ConstructionYearTransformer(BaseEstimator, TransformerMixin):
    def __init__(self,method = 'custom'): 
        self.column_names = [] 
        #self.init_radius = init_radius
        #self.increment_radius = increment_radius
        self.method = method
        pass ##Nothing else to do
        
    def fit(self, X, y=None):
        X['construction_year'] = X['construction_year'].astype(float)
        if self.method == 'custom':
            year_recorded = X[X['construction_year'] > 0]\
                            ['date_recorded'].\
                            apply(lambda x: int(x.split("-")[0]))
            year_constructed = X[X['construction_year'] > 0]['construction_year']
            self.median_age = np.median(year_recorded - year_constructed)
            self.column_names = ['age']
            return self
        if self.method == 'median':
               X['construction_year'] = X['construction_year'].astype(float)
               #X['gps_height'] = X['gps_height'].fillna(0)
               self.median = \
                          np.median(list(X[X['construction_year'] != 0]['construction_year']))
               if math.isnan(self.median):
                  self.median = 0
               self.column_names = ['construction_year']
               return self
        if self.method == 'mean':
               X['construction_year'] = X['construction_year'].astype(float)
               #X['gps_height'] = X['gps_height'].fillna(0)
               self.mean = np.mean(list(X[X['construction_year'] != 0]['construction_year']))
               if math.isnan(self.mean):
                  self.mean = 0
               self.column_names = ['construction_year']
               return self

        if self.method == 'ignore':
               self.column_names = ['construction_year']
               return self
          
    def transform(self,X):
        if self.method == 'custom':
            year_recorded = list(X['date_recorded'].apply(lambda x: int(x.split("-")[0])))
            year_constructed = list(X['construction_year'])
            age = []
            for i,j in enumerate(year_constructed):
                if j == 0:
                   age.append(self.median_age)
                else:
                   temp_age = year_recorded[i] - year_constructed[i]
                   if temp_age < 0:
                      temp_age = self.median_age
                   age.append(temp_age)   
            X['age'] = age
            self.column_names = ['age']
            #self.column_names = X.columns
            return X[['age']]
        if self.method == 'median':      
                X['construction_year'] = X['construction_year'].astype(float)
                X['construction_year'] = X['construction_year'].fillna(0)
                construction_year = np.array(list(X['construction_year']))
                construction_year[construction_year == 0] = self.median
                self.column_names = ['construction_year']
                X['construction_year'] = construction_year
                return X[['construction_year']]

        if self.method == 'mean':
                X['construction_year'] = X['construction_year'].astype(float)
                X['construction_year'] = X['construction_year'].fillna(0)
                construction_year = np.array(list(X['construction_year']))
                construction_year[construction_year == 0] = self.mean
                self.column_names = ['construction_year']
                X['construction_year'] = construction_year
                return X[['construction_year']]
        
        if self.method == 'ignore':      
                X['construction_year'] = X['construction_year'].astype(float)
                X['construction_year'] = X['construction_year'].fillna(0)
                self.column_names = ['construction_year']
                return X[['construction_year']]       

### Test

In [16]:
cyt_test_df = df[['construction_year', 'date_recorded']].copy()

cyt = ConstructionYearTransformer().fit(cyt_test_df)
cyt.transform(cyt_test_df)

,age
0,12.0
1,3.0
2,4.0
3,27.0
4,13.0
...,...
59395,14.0
59396,15.0
59397,13.0
59398,13.0


### HighCardTransformer()

In [53]:
# take columns and turn them into 3 numerical columns representing percent of target
class HighCardTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.column_names = []
        self.df = pd.DataFrame()
        self.mean_map = {}
        pass

    def fit(self, X, y=None):
        X = pd.concat([X, y], axis=1)
        self.target_col = y.name
        dummies_status = pd.get_dummies(X[self.target_col])
        dummies_status.columns = [ f"{self.target_col}_{column_name}" for column_name in dummies_status.columns.tolist()]
        self.dummies = dummies_status.columns
        X = X.join(dummies_status)
        
        for target in self.dummies:
            self.mean_map[target] = {}
            for col in X.columns:
                for val in X[col].unique():
                    mean = np.mean(X[X[col] == val][target])
                    if (np.isnan(mean)) or (val in ['0','-']):
                        mean = 0
                    self.mean_map[target][val] = self.mean_map.get(
                        val, mean)
        self.column_names = X.columns
        return self

    def transform(self, X):
        for feature_name in X.columns:
            for target_col in self.dummies:
                X[feature_name+'_'+target_col[-1:]] = X[feature_name].map(self.mean_map[target_col])
            X = X.drop(feature_name, axis=1)
        self.column_names = X.columns
        return X

In [50]:
df['status_code'] = df['status_group'].astype('category').cat.codes

### Test

In [54]:
hct_test_df = df[['funder', 'installer', 'scheme_name', 'status_code']].sample(100).copy()
hct = HighCardTransformer().fit(hct_test_df.drop('status_code', axis=1), hct_test_df['status_code'])
results = hct.transform(hct_test_df.drop('status_code', axis=1))

In [55]:
results

,funder_0,funder_1,funder_2,installer_0,installer_1,installer_2,scheme_name_0,scheme_name_1,scheme_name_2
38130,1.000000,0.000000,0.000000,0.375,0.15625,0.46875,1.0,0.0,0.0
29666,0.000000,0.000000,0.000000,0.000,0.00000,0.00000,0.0,0.0,0.0
18677,1.000000,0.000000,0.000000,1.000,0.00000,0.00000,0.5,0.0,0.5
51365,0.000000,0.000000,1.000000,0.000,0.00000,1.00000,0.0,0.0,1.0
35526,0.521739,0.086957,0.391304,0.375,0.15625,0.46875,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
10135,0.000000,0.333333,0.666667,0.375,0.15625,0.46875,0.0,0.0,0.0
12916,0.666667,0.000000,0.333333,0.000,0.00000,1.00000,0.0,0.0,0.0
38815,0.625000,0.000000,0.375000,0.500,0.00000,0.50000,0.0,0.0,0.0
57161,1.000000,0.000000,0.000000,1.000,0.00000,0.00000,0.0,0.0,0.0


### DistrictCodeMerge()

In [21]:
# Takes 'region' and 'district_code' and combines them together
class DistrictCodeMerge(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.column_names = []
        pass
    
    def fit(self, X, y=None):
        self.column_names = ['district']
        return self

    def transform(self, X):
        X['district'] = X['region'] + ', ' + X['district_code'].astype(str)
        self.column_names = ['district']
        return X[['district']]

### Test

In [22]:
dcm_test_df = df[['region', 'district_code']].copy()

dcm = DistrictCodeMerge().fit(dcm_test_df)
dcm.transform(dcm_test_df).sample(3)

,district
33570,"Kagera, 1"
21818,"Pwani, 53"
15396,"Ruvuma, 2"


### ExtractionMerge()

In [23]:
# Take in 'extraction_type', 'extraction_type_group', 'extraction_type_class' and flatten them
class ExtractionMerge(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.column_names = []
        pass
    
    def __unique(self, sequence):
        seen = set()
        return [x for x in sequence if not (x in seen or seen.add(x))]
    
    def __merge(self, x):
        x = x.split(',')
        x = self.__unique(x)
        return ":".join(x)
    
    def fit(self, X, y=None):
        self.column_names = ['extraction']
        return self
    
    def transform(self, X):
        X['extraction'] = X['extraction_type_class'] + ',' + X['extraction_type_group'] + ',' + X['extraction_type']
        X['extraction'] = X['extraction'].apply(self.__merge)
        self.column_names = ['extraction']
        return X[[self.column_names]]

### Test

In [24]:
lil_df = df[['extraction_type', 'extraction_type_group', 'extraction_type_class']].copy()

em = ExtractionMerge().fit(lil_df)
em.transform(lil_df).sample(3)

KeyError: "None of [Index([('extraction',)], dtype='object')] are in the [columns]"

### Mean Status of pumps in radius

In [41]:
# will work the same way as the gps imputer. 
class MeanPumpTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, init_radius=0.1, increment_radius=0.3): 
        self.column_names = [] 
        self.init_radius = init_radius
        self.increment_radius = increment_radius
        self.df = pd.DataFrame()
        
    def __get_subset_records(self, latitude, longitude, df, radius):
        latitude_from = latitude - radius
        latitude_to = latitude + radius
        longitude_from = longitude - radius
        longitude_to = longitude + radius
        
        df_temp = df[(df['latitude'] >= latitude_from) & (df['latitude'] <= latitude_to) & \
                  (df['longitude'] >= longitude_from) & (df['longitude'] <= longitude_to)]
        return df_temp
        
    def fit(self, X, y=None):
        self.df['status_code'] = y.astype('category').cat.codes
        self.df['latitude'] = X[X['latitude'] != -2e-08]['latitude'] 
        self.df['longitude'] = X[X['longitude'] != 0]['longitude']
        self.df.sort_values(['latitude', 'longitude'], inplace=True)
        self.column_names = ['latitude', 'longitude']
        return self
    
    def transform(self, X):
        mean_pump_functionality = []
        for latitude, longitude in zip(X['latitude'],X['longitude']):
            radius = self.init_radius

            mean_pump_temp = 0
            while mean_pump_temp == 0 and radius <= 2:
                df_temp = self.__get_subset_records(latitude,longitude,self.df,radius)

                mean_pump_temp = np.mean(df_temp['status_code'])

                radius = self.increment_radius + radius

            mean_pump_functionality.append(mean_pump_temp)
        X['mean_pump_functionality'] = mean_pump_functionality
        self.column_names = X.columns
        return X[['mean_pump_functionality']]

In [44]:
dcm_test_df = df[['latitude', 'longitude', 'status_group']].copy()

dcm = MeanPumpTransformer().fit(dcm_test_df, dcm_test_df['status_group'])
df['mean_pump_functionality'] = dcm.transform(dcm_test_df)

In [45]:
df.groupby('status_group')['mean_pump_functionality'].mean()

status_group
functional                 0.693492
functional needs repair    0.855492
non functional             1.050466
Name: mean_pump_functionality, dtype: float64

In [46]:
df['mean_pump_functionality'].isna().sum()

1812